In [1]:
import os 
import pandas as pd
from collections import defaultdict
import numpy as np 
import glob

In [2]:
sample = 'TCGA-BH-A18V-01A-11R-A12D-07.all' #'TCGA-C8-A12P-01A-11R-A115-07.all' #'TCGA-AO-A0JM-01A-21R-A056-07.all'#'TCGA-A2-A0SX-01A-12R-A084-07.all' #'TCGA-A2-A0D2-01A-21R-A034-07.all'
background = 'commit_d4aee54_GTEXcore'

In [3]:
basefolder = '/cluster/work/grlab/projects/projects2020_OHSU/peptides_generation/v2_v2.5f0752a_conf2_annotFrame_cap0_runs_pya0.17.1/TCGA_Breast_1102'

In [4]:
experiment_folder = os.path.join(basefolder, 'filter_{}/{}'.format(sample, background))

In [5]:
file_biexon_generated = os.path.join(basefolder, 'cohort_mutNone', 'meta_peptide_pooled_pq.tsv')

In [6]:
file_longlist  = os.path.join(experiment_folder, 
                         '{}_{}_{}'.format(sample, 
                                      background.replace('commit_', ''),
                                      'kmer_longlist.tsv'))

In [7]:
file_meta = os.path.join(experiment_folder, 
                         '{}_{}_{}'.format(sample, 
                                      background.replace('commit_', ''),
                                      'kmer_peptides_raw333.tsv'))


In [8]:
delim = '\t'

Step 1. Get the all sample experiement longlist: 
Use cut and unique in bash



Step 2. Get the metadata and peptides: 
Done in bash with the command below 

/GitHub/projects2020_ohsu/eth/peptide_search_format/p20220115_format_peptides_longlist_prior.sh

Step 3. Get the formating of the fasta file, and the experiment map done *within this notebook*



# Preprocess the annotation 
### Input: Annotation

In [9]:
ann_path = '/cluster/work/grlab/projects/projects2020_OHSU/annotation/gencode.v32.annotation.gtf'

In [10]:
# From Immunopepper preprocess.py 
def attribute_item_to_dict(a_item, file_type, feature_type):
    """  From attribute item in annotation file to get corresponding dictionary

    Parameters
    ----------
    a_item: str. attribute item
    file_type: str. Choose from {'gtf', 'gff', 'gff3'}
    feature_type: str. Extract other fields. We only
        consider 'CDS', 'mRNA' and 'transcript'

    Returns
    -------
    gtf_dict: dict. store all the necessary data

    """    
    
    gtf_dict = {}
    if file_type.lower() == 'gtf':
        attribute_list = a_item.split('; ')
        for attribute_pair in attribute_list:
            pair = attribute_pair.split(' ')
            gtf_dict[pair[0]] = pair[1][1:-1]
    elif file_type.lower() == 'gff3':
        attribute_list = a_item.split(';')
        for attribute_pair in attribute_list:
            pair = attribute_pair.split('=')
            gtf_dict[pair[0]] = pair[1]
    elif file_type.lower() == 'gff':
        gff_dict = {}
        attribute_list = a_item.split(';')
        for attribute_pair in attribute_list:
            pair = attribute_pair.split('=')
            gff_dict[pair[0]] = pair[1]  # delete "", currently now work on level 2
        if feature_type == 'CDS':
            gtf_dict['transcript_id'] = gff_dict['Parent']
        elif feature_type in {'mRNA', 'transcript'}:  # mRNA or transcript
            gtf_dict['gene_id'] = gff_dict['geneID']
            gtf_dict['transcript_id'] = gff_dict['ID']
            gtf_dict['gene_type'] = gff_dict['gene_type']
            gtf_dict['transcript_type'] = gff_dict['transcript_type']

    return gtf_dict

In [11]:
#  From Immunopepper preprocess.py 
def leq_strand(coord1, coord2, strand):
    if strand == "+":
        return coord1 <= coord2
    else:
        return coord1 >= coord2


In [12]:
# From Immunopepper preprocess.py 
transcript_to_gene_dict = {}    # transcript -> gene id


gene_to_transcript_dict = {}    # gene_id -> list of transcripts
gene_cds_begin_dict = {}        # gene -> list of first CDS exons

transcript_to_cds_dict = {}     # transcript -> list of CDS exons
transcript_cds_begin_dict = {}  # transcript -> first exon of the CDS
transcript_to_strand = {}

file_type = ann_path.split('.')[-1]
chromesome_set = set()
# collect information from annotation file
for line in open(ann_path, 'r'):
    if line[0] == '#':
        continue
    item = line.strip().split('\t')
    chromesome_set.add(item[0])
    feature_type = item[2]
    attribute_item = item[-1]
    attribute_dict = attribute_item_to_dict(attribute_item, file_type, feature_type)
    # store relationship between gene ID and its transcript IDs
    if feature_type in ['transcript', 'mRNA']:
        gene_id = attribute_dict['gene_id']
        transcript_id = attribute_dict['transcript_id']
        if attribute_dict['gene_type'] != 'protein_coding' or attribute_dict['transcript_type']  != 'protein_coding':
            continue
        assert (transcript_id not in transcript_to_gene_dict)
        transcript_to_gene_dict[transcript_id] = gene_id
        if gene_id in gene_to_transcript_dict and transcript_id not in gene_to_transcript_dict[gene_id]:
            gene_to_transcript_dict[gene_id].append(transcript_id)
        else:
            gene_to_transcript_dict[gene_id] = [transcript_id]
        # Todo python is 0-based while gene annotation file(.gtf, .vcf, .maf) is one based
    elif feature_type == "CDS":
        parent_ts = attribute_dict['transcript_id']
        strand_mode = item[6]
        cds_left = int(item[3])-1
        cds_right = int(item[4])
        frameshift = int(item[7])
        transcript_to_strand[parent_ts] = strand_mode
        if parent_ts in transcript_to_cds_dict:
            transcript_to_cds_dict[parent_ts].append((cds_left, cds_right, frameshift))
        else:
            transcript_to_cds_dict[parent_ts] = [(cds_left, cds_right, frameshift)]
        if strand_mode == "+" :
            cds_start, cds_stop = cds_left, cds_right
        else:
            cds_start, cds_stop = cds_right, cds_left

        # we only consider the start of the whole CoDing Segment
        if parent_ts not in transcript_cds_begin_dict or \
           leq_strand(cds_start, transcript_cds_begin_dict[parent_ts][0], strand_mode):
            transcript_cds_begin_dict[parent_ts] = (cds_start, cds_stop, item)

# collect first CDS exons for all transcripts of a gene
for ts_key in transcript_to_gene_dict:
    target_gene = transcript_to_gene_dict[ts_key]
    if target_gene not in gene_cds_begin_dict:
        gene_cds_begin_dict[target_gene] = []
    if ts_key in transcript_cds_begin_dict:
        gene_cds_begin_dict[target_gene].append(transcript_cds_begin_dict[ts_key])


In [13]:
# NEED TO SORT THE TRANSCRIPT TO CDS 

In [14]:
# Custom collection of CDS 
transcript_cds_begin_dict_bis = {}
transcript_cds_end_dict_bis = {}

gene_cds_begin_dict_bis = defaultdict(list)
gene_cds_end_dict_bis = defaultdict(list)

# will be in reading order 
for ts_key in transcript_to_cds_dict:
    if transcript_to_strand[ts_key] == '+': # '+'
        transcript_cds_begin_dict_bis[ts_key] = (transcript_to_cds_dict[ts_key][0][0],
                                                 transcript_to_cds_dict[ts_key][0][1], '+')
        transcript_cds_end_dict_bis[ts_key] = (transcript_to_cds_dict[ts_key][-1][0],
                                                 transcript_to_cds_dict[ts_key][-1][1], '+')
        

    else: 
        transcript_cds_begin_dict_bis[ts_key] = (transcript_to_cds_dict[ts_key][0][1],
                                                 transcript_to_cds_dict[ts_key][0][0], '-')
        transcript_cds_end_dict_bis[ts_key] = (transcript_to_cds_dict[ts_key][-1][1],
                                                 transcript_to_cds_dict[ts_key][-1][0], '-')
    
    assert(transcript_cds_begin_dict_bis[ts_key][0] == transcript_cds_begin_dict[ts_key][0])
    assert(transcript_cds_begin_dict_bis[ts_key][1] == transcript_cds_begin_dict[ts_key][1])

# collect first, last CDS exons for all transcripts of a gene
for ts_key in transcript_to_gene_dict:
    target_gene = transcript_to_gene_dict[ts_key]
    gene_cds_begin_dict_bis[target_gene].append(transcript_cds_begin_dict_bis[ts_key])
    gene_cds_end_dict_bis[target_gene].append(transcript_cds_end_dict_bis[ts_key])

# Step 3. Get the experiement/ kmer map 

## Step a. Get the experiment IDS
### Input: J_experiment_map file

In [16]:
experiement_files = glob.glob(experiment_folder + '/*Uniprot*tsv')

In [17]:
folder_OHSU = '/cluster/work/grlab/projects/projects2020_OHSU/share_OHUS_PNLL/Aug21_graph_data_updatedfilters'

In [18]:
exp_map = pd.read_csv(os.path.join(folder_OHSU, 'J_experiment_map.tsv'), sep = '\t')

In [19]:
exp_map.head()

,id,motif_filter,min_sample_reads,#_of_cohort_samples,reads_per_cohort_sample,normal_cohort_id,#_normal_samples_allowed,reads_per_normal_sample
0,1,1,0,0,0,paired,0,0
1,2,1,0,0,0,paired,2,3
2,3,1,0,0,0,paired,2,10
3,4,1,0,0,0,paired,2,-1
4,5,1,0,0,0,paired,10,3


In [20]:
def path_to_condition(all_paths):
    '''Converts ETH Names into experimental conditions '''
    # None handling added 
    
    #all_paths = [path.replace('None', '-1') for path in all_paths]
    sample_expr = [os.path.basename(path).split('ref_SampleLim')[1].split('Cohort')[0]
                   for path in all_paths ]

    foreground_cohort_expr = [os.path.basename(path).split('CohortLim')[1].split('Across')[0]
                                       for path in all_paths ]

    foreground_cohort_samples = [os.path.basename(path).split('Across')[1].split('_Filt')[0] 
                                 for path in all_paths ]

    background_cohort_expr = [os.path.basename(path).split('Cohortlim')[1].split('Across')[0]
                              for path in all_paths ]

    background_cohort_samples = [os.path.basename(path).split('Across')[2].split('_FiltUn')[0]
                                 for path in all_paths ]
    background_cohort_id = [os.path.basename(path).split('Normals')[1].split('lim')[0] for path in all_paths ]

    legend_quant = pd.DataFrame({'min_sample_reads':sample_expr, 
                  'reads_per_cohort_sample': foreground_cohort_expr, 
                  '#_of_cohort_samples': foreground_cohort_samples, 
                  'reads_per_normal_sample': background_cohort_expr,
                  '#_normal_samples_allowed' : background_cohort_samples, 
                    'normal_cohort_id':background_cohort_id, 
                    'original_name':all_paths})

    return legend_quant

In [21]:
experiement_files

['/cluster/work/grlab/projects/projects2020_OHSU/peptides_generation/v2_v2.5f0752a_conf2_annotFrame_cap0_runs_pya0.17.1/TCGA_Breast_1102/filter_TCGA-BH-A18V-01A-11R-A12D-07.all/commit_d4aee54_GTEXcore/G_TCGA-BH-A18V-01A-11R-A12D-07.all_ref_SampleLim2.0CohortLim1.0Across10_FiltNormalsGtexcoreCohortlim3.0AcrossNone_FiltUniprot.tsv',
 '/cluster/work/grlab/projects/projects2020_OHSU/peptides_generation/v2_v2.5f0752a_conf2_annotFrame_cap0_runs_pya0.17.1/TCGA_Breast_1102/filter_TCGA-BH-A18V-01A-11R-A12D-07.all/commit_d4aee54_GTEXcore/G_TCGA-BH-A18V-01A-11R-A12D-07.all_ref_SampleLim0.0CohortLim5.0Across2_FiltNormalsGtexcoreCohortlim0.0Across0_FiltUniprot.tsv',
 '/cluster/work/grlab/projects/projects2020_OHSU/peptides_generation/v2_v2.5f0752a_conf2_annotFrame_cap0_runs_pya0.17.1/TCGA_Breast_1102/filter_TCGA-BH-A18V-01A-11R-A12D-07.all/commit_d4aee54_GTEXcore/G_TCGA-BH-A18V-01A-11R-A12D-07.all_ref_SampleLim2.0CohortLim0.0Across1_FiltNormalsGtexcoreCohortlimNoneAcross2_FiltUniprot.tsv',
 '/clust

In [22]:
path_to_table = path_to_condition(experiement_files)


In [23]:
path_to_table.loc[path_to_table['min_sample_reads'] == 'None', 
                  'min_sample_reads'] = 0
path_to_table.loc[path_to_table['reads_per_cohort_sample'] == 'None', 
                  'reads_per_cohort_sample'] = 0
path_to_table.loc[path_to_table['#_of_cohort_samples'] == 'None', 
                  '#_of_cohort_samples'] = 0
path_to_table.loc[path_to_table['reads_per_normal_sample'] == 'None', 
                  'reads_per_normal_sample'] = -1
path_to_table.loc[path_to_table['#_normal_samples_allowed'] == 'None', 
                  '#_normal_samples_allowed'] = -1

In [24]:
path_to_table.head()

,min_sample_reads,reads_per_cohort_sample,#_of_cohort_samples,reads_per_normal_sample,#_normal_samples_allowed,normal_cohort_id,original_name
0,2.0,1.0,10,3.0,-1,GtexcoreCohort,/cluster/work/grlab/projects/projects2020_OHSU...
1,0.0,5.0,2,0.0,0,GtexcoreCohort,/cluster/work/grlab/projects/projects2020_OHSU...
2,2.0,0.0,1,-1,2,GtexcoreCohort,/cluster/work/grlab/projects/projects2020_OHSU...
3,0.0,1.0,2,3.0,-1,GtexcoreCohort,/cluster/work/grlab/projects/projects2020_OHSU...
4,2.0,5.0,1,0.0,0,GtexcoreCohort,/cluster/work/grlab/projects/projects2020_OHSU...


In [25]:
exp_map.loc[exp_map['normal_cohort_id'] == 'core_GTEx', 'normal_cohort_id'] = 'GtexcoreCohort'

In [26]:
path_to_table['motif_filter'] = 0 

In [27]:
for col in path_to_table.columns: 
    if col != 'original_name':
        print(col)
        print(exp_map[col].dtypes)
        print(path_to_table[col].dtypes)
        if (exp_map[col].dtypes != path_to_table[col].dtypes) :
            print('change type')
            if exp_map[col].dtypes == 'int64':
                path_to_table[col] = path_to_table[col].astype( 'float64' )
            path_to_table[col] = path_to_table[col].astype( exp_map[col].dtypes ) 
        print('\n')

min_sample_reads
int64
object
change type


reads_per_cohort_sample
int64
object
change type


#_of_cohort_samples
int64
object
change type


reads_per_normal_sample
int64
object
change type


#_normal_samples_allowed
int64
object
change type


normal_cohort_id
object
object


motif_filter
int64
int64




In [28]:
exp_map.columns

Index(['id', 'motif_filter', 'min_sample_reads', '#_of_cohort_samples',
       'reads_per_cohort_sample', 'normal_cohort_id',
       '#_normal_samples_allowed', 'reads_per_normal_sample'],
      dtype='object')

In [29]:
path_to_table.columns

Index(['min_sample_reads', 'reads_per_cohort_sample', '#_of_cohort_samples',
       'reads_per_normal_sample', '#_normal_samples_allowed',
       'normal_cohort_id', 'original_name', 'motif_filter'],
      dtype='object')

In [30]:
id_mapped = path_to_table.merge(exp_map, on = ['min_sample_reads', 'reads_per_cohort_sample', \
                                   '#_of_cohort_samples', 'reads_per_normal_sample', \
                                   '#_normal_samples_allowed',
                                   'normal_cohort_id', 'motif_filter'], how = 'inner')

In [31]:
id_mapped['filename'] = [os.path.basename(name) for name in id_mapped['original_name']]

In [32]:
id_mapped.shape

(120, 10)

In [33]:
id_mapped.head()

,min_sample_reads,reads_per_cohort_sample,#_of_cohort_samples,reads_per_normal_sample,#_normal_samples_allowed,normal_cohort_id,original_name,motif_filter,id,filename
0,2,1,10,3,-1,GtexcoreCohort,/cluster/work/grlab/projects/projects2020_OHSU...,0,474,G_TCGA-BH-A18V-01A-11R-A12D-07.all_ref_SampleL...
1,0,5,2,0,0,GtexcoreCohort,/cluster/work/grlab/projects/projects2020_OHSU...,0,397,G_TCGA-BH-A18V-01A-11R-A12D-07.all_ref_SampleL...
2,2,0,1,-1,2,GtexcoreCohort,/cluster/work/grlab/projects/projects2020_OHSU...,0,430,G_TCGA-BH-A18V-01A-11R-A12D-07.all_ref_SampleL...
3,0,1,2,3,-1,GtexcoreCohort,/cluster/work/grlab/projects/projects2020_OHSU...,0,396,G_TCGA-BH-A18V-01A-11R-A12D-07.all_ref_SampleL...
4,2,5,1,0,0,GtexcoreCohort,/cluster/work/grlab/projects/projects2020_OHSU...,0,439,G_TCGA-BH-A18V-01A-11R-A12D-07.all_ref_SampleL...


In [34]:
# path_to_table.loc[(path_to_table['min_sample_reads'] == 0) & \
#             (path_to_table['reads_per_cohort_sample'] == 0) & \
#            (path_to_table['#_of_cohort_samples'] == 2) & \
#            (path_to_table['reads_per_normal_sample'] == -1) & \
#            (path_to_table['#_normal_samples_allowed'] == 2) & \
#            (path_to_table['normal_cohort_id'] == 'GtexcoreCohort') ]

## Step b. Get the longlist of peptides
### Input: Peptide files from ETH filtering

In [35]:
kmer_file_idx = defaultdict(list) 
for io, eth_path in enumerate(experiement_files):
    eth_path = glob.glob(eth_path + '/*part*')
    if eth_path:
        eth_path = eth_path[0]
        df_eth = pd.read_csv(eth_path, sep="\t", usecols = ['kmer'])
        df_eth = set(df_eth['kmer'])
        base_name = eth_path.split('/')[-2]
        idx_exp = id_mapped.loc[id_mapped['filename'] == base_name, 'id'].values[0]
        for kmer in df_eth:
            kmer_file_idx[kmer].append(str(idx_exp))

# Step 4. Format the peptide raw file  
### Input:  file meta with kmers and bi-exons matching the longlist of the samples 

In [54]:
file_meta = os.path.join(experiment_folder,
                         '{}_{}_{}'.format(sample, 
                                      background.replace('commit_', ''),
                                     'kmer_peptides_raw.tsv'))

In [55]:
file_save = os.path.join(experiment_folder,
                         'G_{}_{}'.format(sample.split('.')[0], 
                                      'query_peptides.fa'))

In [56]:
file_save_experiement= os.path.join(experiment_folder,
                         'G_{}_{}'.format(sample.split('.')[0], 
                                      'experiments_per_peptide.tsv'))

In [57]:
fields_meta_peptide_dict = ['peptide', 'id', 'readFrame', 'geneName', 'geneChr', 'geneStrand',
                                'mutationMode',
                                'junctionAnnotated', 'hasStopCodon', 'isInJunctionList',
                                'isIsolated', 'variantComb', 'variantSegExpr', 'modifiedExonsCoord',
                                'originalExonsCoord', 'vertexIdx',
                                'kmerType']



In [58]:
def extract_peptide_fields(pep):
    '''0 peptide
    1 id
    2 readFrame
    3 geneName
    4 geneChr
    5 geneStrand
    6 mutationMode
    7 junctionAnnotated
    8 hasStopCodon
    9 isInJunctionList
    10 isIsolated
    11 variantComb
    12 variantSegExpr
    13 modifiedExonsCoord
    14 originalExonsCoord
    15 vertexIdx
    16 kmerType '''
    pep = pep.split(',')
    pep_seq = pep[0]
    pep_gene = pep[3]
    pep_orig_coordS = pep[14].split('/')
    pep_modif_coordS = pep[13].split('/')
    strand = pep[5]
    has_stop_codon = pep[8]
    return pep, pep_seq, pep_gene, pep_orig_coordS, pep_modif_coordS, strand, \
        has_stop_codon

In [59]:
def extract_end_starts(pep_orig_coord):
    ''' Get peptide end and start coordinates'''
    if strand == '+': # Do - strand 
        pep_start = np.int(pep_orig_coord[0])
        pep_end = np.int(pep_orig_coord[-1])
    else: 
        pep_start = np.int(pep_orig_coord[1])
        pep_end = np.int(pep_orig_coord[-2])
    return pep_start, pep_end

In [60]:
def get_include_flag(start_cds, end_cds, pep_start, pep_end, has_stop_codon ):
    '''Use end and start coordinates for 3' 5' include flag'''
    if pep_start in start_cds: # We will always miss things that are new in the graph 
        pep_5include = 1
    else: 
        pep_5include = 0 
    if (pep_end in end_cds) or (has_stop_codon) == '1':
        pep_3include = 1
    else: 
        pep_3include = 0 
    return pep_5include, pep_3include

In [61]:
def get_nt_len_with_aa_shift(pep_modi_coord):
    '''Get nt length of each exon involved -> jx_list, shift_list'''
    tot_len = 0 
    shift = 0 
    jx_list = []
    jx_list_ori = []
    shift_list = []
    for pair in np.arange(0, len(pep_modi_coord), 2):
        cds = int(pep_modi_coord[pair + 1]) - int(pep_modi_coord[pair])  # 0 based, open right 
        jx_list_ori.append(cds)
        cds += shift 
        shift = cds % 3
        jx_list.append(cds - shift)
        shift_list.append(shift)
        
    return jx_list, shift_list, jx_list_ori

In [62]:
def get_aaPos_betweenFlag(shift_list, jx_list):
    '''Get aa position of the junction
    the junction coordinate jx_pos is the 0-based position in the peptide 
    of the amino acid that either overlaps the junction (if the junction is 
    in the middle of a codon), or is immediately before it if the junction 
    occurs between codons'''
    if shift_list[0]: # junction is inside an amino acid
        aa_junction_pos0 = int((jx_list[0] / 3)) # because 0 based
        between_codons0 = 0 
    else: # junction is between amino acids 
        aa_junction_pos0 = int((jx_list[0] / 3) - 1)  # because 0 based
        between_codons0 = 1
    if len(shift_list) > 2:
        if shift_list[1]:
            aa_junction_pos1 = int((jx_list[1] / 3)) # because 0 based
            between_codons1 = 0 
        else: 
            aa_junction_pos1 = int((jx_list[1] / 3) - 1)  # because 0 based
            between_codons1 = 1 
    else:
        aa_junction_pos1 = None
        between_codons1 = None
        
    return aa_junction_pos0, between_codons0, aa_junction_pos1, between_codons1

In [63]:
def get_genomic_coordinates(pep_modi_coord):
    '''We have in + case: exon1_start, exon 1_stop, exon2_start, exon2_stop, exon3_start, exon3_stop
     In the - case: exon1_stop, exon 1_start, exon 2_stop, exon2_start, exon3_stop, exon3_start'''
    genome_junction_pos1 = None
    if strand == '+':
        genome_junction_pos0 = '{}_{}'.format(pep_modi_coord[1], pep_modi_coord[2])
        if len(pep_modi_coord) > 4:
            genome_junction_pos1 = '{}_{}'.format(pep_modi_coord[3], pep_modi_coord[4])
    else:
        genome_junction_pos0 = '{}_{}'.format(pep_modi_coord[0], pep_modi_coord[3])
        if len(pep_modi_coord) > 4:
            genome_junction_pos1 = '{}_{}'.format(pep_modi_coord[2], pep_modi_coord[5])
    return genome_junction_pos0, genome_junction_pos1

In [64]:
def split_coord(pep_coord):
    pep_coord = pep_coord.split(';')
    pep_coord = [coord for coord in pep_coord if (coord != 'None') and (coord != 'nan')]
    return pep_coord

In [65]:
def preprocess_line(line):
    line = line.replace('3-exons_9-mer ', '3-exons_9-mer@').replace('2-exons ', '2-exons@')
    kmer = line.split(',')[0]
    peptides = ','.join(line.split(',')[1:])
    peptides = peptides.split('@')
    return line, kmer, peptides

In [66]:
def write_peptide_to_experiment(filepointer, pep_idx=None, pep_seq=None,\
                                idx=None, header=False):
    if header:
        header_exp = 'peptide_id\tpeptide_sequence\texperiment_ids\n'
        filepointer.write(header_exp)
    elif pep_idx is not None:
        exp_line = '{}\t{}\t{}\n'.format(pep_idx, 
                                         pep_seq,
                                         idx)
        filepointer.write(exp_line)


In [88]:
def write_fasta(filepointer, pep_idx, aa_junction_pos, 
                aa_junction_pos1, between_codons, between_codons1,
                pep_5include, pep_3include, pep_gene, 
                genome_junction_pos, genome_junction_pos1, 
                kmer, jx_pep1, jx_pep2, do_write=True):
        
    pep_handle1 = '>pepID-{};jx_pos-{};between_codons-{};includes_5\'-{};includes_3\'-{};gene-{};jx_coord-{}'.format(
    pep_idx, aa_junction_pos, between_codons, pep_5include, 
    pep_3include, pep_gene, genome_junction_pos)

    pep_handle2 = '>pepID-{};jx_pos-{};between_codons-{};includes_5\'-{};includes_3\'-{};gene-{};jx_coord-{}'.format(
    pep_idx, aa_junction_pos1, between_codons1, pep_5include, 
    pep_3include, pep_gene, genome_junction_pos1)

    if kmer in jx_pep1:
        pep_idx+=1
        sp.write(pep_handle1 + '\n')
        sp.write(pep_seq + '\n')
    elif kmer in jx_pep2:
        pep_idx+=1
        sp.write(pep_handle2 + '\n')
        sp.write(pep_seq + '\n')
    else:
        pep_idx+=1
        sp.write(pep_handle1 + '\n')
        sp.write(pep_seq + '\n')
        pep_idx+=1
        sp.write(pep_handle2 + '\n')
        sp.write(pep_seq + '\n')
    return pep_idx

In [110]:
def cut_peptides(jx_list, between_codons, between_codons1, aa_junction_pos, 
                 aa_junction_pos1, print_ = False):
    peptide_cut = []
    aa_junction_pos_shift = aa_junction_pos + 1 
    exon1, aa_jx1, exon2, aa_jx2, exon3 = '', '', '', '', ''

    if len(jx_list) == 2:
        if between_codons:
            exon1 = pep_seq[:aa_junction_pos_shift]
            exon2 = pep_seq[aa_junction_pos_shift:]
        else:
            exon1 = pep_seq[:aa_junction_pos]
            aa_jx1 = pep_seq[aa_junction_pos:aa_junction_pos_shift]
            exon2 = pep_seq[aa_junction_pos_shift:]
    elif len(jx_list) == 3:
        aa_junction2_pos_shift =  aa_junction_pos1 + aa_junction_pos + 1
        aa_junction2_pos_sshift = aa_junction_pos1 + aa_junction_pos + 2
        if between_codons and between_codons1: 
            exon1 = pep_seq[:aa_junction_pos_shift]
            exon2 = pep_seq[aa_junction_pos_shift:
                          aa_junction2_pos_sshift]
            exon3 = pep_seq[aa_junction2_pos_sshift:]
        if (not between_codons) and between_codons1: 
            exon1 = pep_seq[:aa_junction_pos ]
            aa_jx1 = pep_seq[aa_junction_pos:aa_junction_pos_shift]
            exon2 = pep_seq[aa_junction_pos_shift:
                          aa_junction2_pos_sshift]
            exon3 = pep_seq[aa_junction2_pos_sshift:]
        if (between_codons) and (not between_codons1):
            exon1 = pep_seq[:aa_junction_pos_shift]
            exon2 = pep_seq[aa_junction_pos_shift :
                          aa_junction2_pos_shift]
            aa_jx2 = pep_seq[aa_junction2_pos_shift:
                         aa_junction2_pos_sshift]
            exon3 = pep_seq[aa_junction2_pos_sshift:]
        if (not between_codons) and (not between_codons1):
            exon1 = pep_seq[:aa_junction_pos ]
            aa_jx1 = pep_seq[aa_junction_pos:aa_junction_pos_shift]
            exon2 = pep_seq[aa_junction_pos_shift :
                          aa_junction2_pos_shift]
            aa_jx2 = pep_seq[aa_junction2_pos_shift:
                         aa_junction2_pos_sshift]
            exon3 = pep_seq[aa_junction2_pos_sshift:]
    if print_:
        print(f'exon1:{exon1}, aa_jx1:{aa_jx1}, exon2:{exon2}, aa_jx2:{aa_jx2}, exon3:{exon3} \n')
    return exon1 + aa_jx1 + exon2, exon2 + aa_jx2 + exon3 

In [111]:
def print_stats(print_, kmer, pep_seq, strand, pep_orig_coord, pep_modi_coord, jx_list,
               jx_list_ori, genome_junction_pos, genome_junction_pos1,
               aa_junction_pos, aa_junction_pos1, between_codons, between_codons1):
    
    if print_:
        print(kmer, pep_seq, strand, '/',';'.join(pep_orig_coord), '/modif',
                                  ';'.join(pep_modi_coord), '/', jx_list, 
                                  '/orig', jx_list_ori, 
                                  genome_junction_pos, genome_junction_pos1)
        print(aa_junction_pos, between_codons, aa_junction_pos1, between_codons1)
        print('pep len', len(pep_seq))


In [112]:
# Note: corrected the inversion of coordinates, extracted the right matching coordinate position for 3 exon peptide, remove has_stop_codon == 0 break 

In [114]:
## Format and create flags 
pep_idx = 0 
kmer_len = 9 
#sp = 'None'
print_ = False
write_=True
with open(file_save_experiement, 'w') as ep: # experiment file for sample
    write_peptide_to_experiment(ep, header=True)
    with open(file_save, 'w') as sp: # peptide file for sample
        with open(file_meta, 'r') as fp:
            print("open {}".format(file_meta))
            lines = fp.readlines()
            ### Iterate over kmers 
            for line in lines:
                if 'modifiedExonsCoord' in line: #header
                    continue
                # --- TEST ---
    #             if ('MSDTWSS' in line): # TODO REMOVE TEST LHAGSGR 'MSDTWSS!!!'YSCMLLWLI
    #                 print(line)
                # --- END TEST ---
                line, kmer, peptides = preprocess_line(line)
                #Iterate over kmer-associated peptides
                for pep in peptides:
                    pep, pep_seq, pep_gene, pep_orig_coordS, pep_modif_coordS, \
                    strand, has_stop_codon = extract_peptide_fields(pep)
#                     if has_stop_codon == '0':
#                         break 

                    # Iterate over original genomic coordinates for include flag
                    for pep_orig_coord in pep_orig_coordS:
                        pep_orig_coord = split_coord(pep_orig_coord)
                        start_cds = [ first_exon[0] for first_exon in 
                                     gene_cds_begin_dict_bis[pep_gene] ] 
                        end_cds = [ last_exon[1] for last_exon in 
                                   gene_cds_end_dict_bis[pep_gene] ] 
                        pep_start, pep_end = extract_end_starts(pep_orig_coord)
                        pep_5include, pep_3include = get_include_flag(start_cds, end_cds, 
                                                                      pep_start, pep_end, 
                                                                      has_stop_codon )

                    # Iterate over modified genomic coordinates for the 
                    # junction position flag and between codon flag and genomic coordinates
                    for pep_modi_coord in pep_modif_coordS:
                        pep_modi_coord = split_coord(pep_modi_coord)
                        if len(pep_modi_coord) <=2:
                            continue
                        jx_list, shift_list, jx_list_ori = \
                            get_nt_len_with_aa_shift(pep_modi_coord)
                        aa_junction_pos, between_codons, aa_junction_pos1, between_codons1 = \
                            get_aaPos_betweenFlag(shift_list,jx_list)
                        genome_junction_pos, genome_junction_pos1 = \
                            get_genomic_coordinates(pep_modi_coord)

                        # --- TEST ---
    #                     if ('MSDTWSS' not in pep_seq): # TODO REMOVE TEST LHAGSGR 'MSDTWSS!!!'YSCMLLWLI
    #                         continue
    #                         # 'MSDTWSS!!!', 'NAWSRKKRAHL' same issue..
                        # --- END TEST ---
                        print_stats(print_, kmer, pep_seq, strand, pep_orig_coord, 
                                    pep_modi_coord, jx_list, jx_list_ori, genome_junction_pos,
                                    genome_junction_pos1, aa_junction_pos, aa_junction_pos1, 
                                    between_codons, between_codons1)
                        jx_pep1, jx_pep2 = cut_peptides(jx_list, between_codons,
                                                        between_codons1, aa_junction_pos, 
                                                         aa_junction_pos1, print_=print_)


                        pep_idx = write_fasta(sp, pep_idx, aa_junction_pos,
                                    aa_junction_pos1, between_codons, 
                                    between_codons1, pep_5include, pep_3include, pep_gene, 
                                    genome_junction_pos, genome_junction_pos1, 
                                    kmer, jx_pep1, jx_pep2, do_write=write_)

                        write_peptide_to_experiment(ep, pep_idx, pep_seq,';'.join(
                            kmer_file_idx[kmer])) 


print(file_save_experiement)
print(file_save)


open /cluster/work/grlab/projects/projects2020_OHSU/peptides_generation/v2_v2.5f0752a_conf2_annotFrame_cap0_runs_pya0.17.1/TCGA_Breast_1102/filter_TCGA-BH-A18V-01A-11R-A12D-07.all/commit_d4aee54_GTEXcore/TCGA-BH-A18V-01A-11R-A12D-07.all_d4aee54_GTEXcore_kmer_peptides_raw.tsv
/cluster/work/grlab/projects/projects2020_OHSU/peptides_generation/v2_v2.5f0752a_conf2_annotFrame_cap0_runs_pya0.17.1/TCGA_Breast_1102/filter_TCGA-BH-A18V-01A-11R-A12D-07.all/commit_d4aee54_GTEXcore/G_TCGA-BH-A18V-01A-11R-A12D-07_experiments_per_peptide.tsv
/cluster/work/grlab/projects/projects2020_OHSU/peptides_generation/v2_v2.5f0752a_conf2_annotFrame_cap0_runs_pya0.17.1/TCGA_Breast_1102/filter_TCGA-BH-A18V-01A-11R-A12D-07.all/commit_d4aee54_GTEXcore/G_TCGA-BH-A18V-01A-11R-A12D-07_query_peptides.fa


In [109]:
aa_junction_pos

12

In [76]:
kmer in pep_seq

True

In [72]:
kmer

'AAAALQQHS'

In [78]:
kmer in jx_pep1

False

In [74]:
jx_pep2

'HSSVFSQGWLQNYMIRLDLWMKMYSF'

In [73]:
jx_pep1

'ALGVQQPSLLGASPTIYTQQTALAAAGLTTQTPANYQLTQTAALQQQAAAAAAALQQH'

In [ ]:
pep_modi_coord

In [ ]:
#TODO (1) print as external function (2) match the kmer to one of them (append the before + aa + after // after + aa + after2 and match to the kmer to output the junction position)